In [14]:
import csv
import pandas as pd
from tqdm import tqdm
from requests.compat import urljoin
import requests
import glob
import os
import re
from bs4 import BeautifulSoup

In [2]:
# !python -m spacy download en_core_web_sm

In [39]:
# !pip install spacy
import spacy

nlp = spacy.load("en_core_web_sm")

In [3]:
# Open types.txt and read the types
with open('../data/gold_standard_corpus.tsv', 'r') as f:
    pmcids = f.read().splitlines()

In [14]:
pmcids

['PMC4792959',
 'PMC4556948',
 'PMC5993813',
 'PMC3174205',
 'PMC5962829',
 'PMC3874094',
 'PMC3792120',
 'PMC4901335',
 'PMC3581133',
 'PMC2935479',
 'PMC5225553',
 'PMC5744400',
 'PMC3281816',
 'PMC3583137',
 'PMC4022742',
 'PMC3542345',
 'PMC4452330',
 'PMC4464872',
 'PMC4872455',
 'PMC3651197',
 'PMC3362782',
 'PMC5817132',
 'PMC4313693',
 'PMC4489904',
 'PMC4552872',
 'PMC5376652',
 'PMC5070310',
 'PMC5921292',
 'PMC5641157',
 'PMC3751948',
 'PMC5472290',
 'PMC4649626',
 'PMC5502978',
 'PMC4767726',
 'PMC3897916',
 'PMC5087830',
 'PMC3585192',
 'PMC5484670',
 'PMC5259676',
 'PMC3024232',
 'PMC3097211',
 'PMC5317055',
 'PMC3648400',
 'PMC5750880',
 'PMC5100220',
 'PMC4749753',
 'PMC5344356',
 'PMC5110973',
 'PMC5708618',
 'PMC3598673',
 'PMC3751959',
 'PMC5131611',
 'PMC5891595',
 'PMC3950279',
 'PMC5972578',
 'PMC5082793',
 'PMC5487420',
 'PMC5106849',
 'PMC3858553',
 'PMC3613406',
 'PMC4167147',
 'PMC3599585',
 'PMC2761781',
 'PMC3899050',
 'PMC2481430',
 'PMC5006041',
 'PMC47908

In [6]:
import requests
from urllib.parse import urljoin
import csv
from tqdm import tqdm

def get_Json_through_PMCID(pmcid, provider):
    base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
    article_url = urljoin(base_url, 
                          f"annotationsByArticleIds?articleIds=PMC%3A{pmcid}&provider={provider}&format=JSON")
    r = requests.get(article_url)

    if r.status_code == 200:
        return r
    else:
        return False

def get_epmc_annotations_to_file(PMCids):
    with open('../data/annotations_from_api.csv', 'w', newline='\n') as f1:
        test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

        for each_id in tqdm(PMCids):
            # Loop through each provider
            for provider in ["Europe PMC", "PheneBank"]:
                json_annotations = get_Json_through_PMCID(each_id[3:], provider)  # Just the number is needed, so remove the PMC from the front
                if json_annotations:
                    json_results = json_annotations.json()
                    try:
                        pmc_id = json_results[0]['pmcid']
                        for each_annotation in json_results[0]['annotations']:
                            exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
                            token = each_annotation['tags'][0]['name']
                            ner = each_annotation['type']
                            row = [pmc_id, exact, token, ner, provider]
                            test_writer.writerow(row)
                    except(IndexError):
                        print('No annotations found for ' + str(each_id) + ' from ' + provider)
                else:
                    print('No annotations from ' + provider + ' for ' + str(each_id))
                    continue

# Usage example:
# PMCids = ['PMC123456', 'PMC654321', ...]
# get_epmc_annotations_to_file(PMCids)


In [28]:
# def get_Json_through_PMCID(pmcid):
#     base_url = "https://www.ebi.ac.uk/europepmc/annotations_api/"
#     article_url = urljoin(base_url,
#                           "annotationsByArticleIds?articleIds=PMC%3A" + pmcid + "&provider=Europe%20PMC&format=JSON")
#     r = requests.get(article_url)

#     if r.status_code == 200:
#         return r
#     else:
#         return False


In [32]:
# def get_epmc_annotations_to_file(PMCids):
#     with open('../data/annotations_api.csv', 'w', newline='\n') as f1:
#         test_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')

#         # count = 0
#         for each_id in tqdm(PMCids):
#             # count = count+1
#             # print(each_test_pmc_id + '\t' + str(count))
#             json_annotations = get_Json_through_PMCID(each_id[3:])  # Just the number is needed. So remove the PMC from the front
#             if json_annotations:
#                 json_results = json_annotations.json()
#                 try:
#                     pmc_id = json_results[0]['pmcid']
#                     # print(pmc_id)
#                     for each_annotation in json_results[0]['annotations']:
#                         exact = each_annotation['prefix'] + each_annotation['exact'] + each_annotation['postfix']
#                         token = each_annotation['tags'][0]['name']
#                         ner = each_annotation['type']
#                         row = [pmc_id, exact, token, ner]
#                         test_writer.writerow(row)
#                 except(IndexError):
#                     print('no annotations found!! '+str(each_id))
#             else:
#                 print('no annotations! '+str(each_id))
#                 continue


In [7]:
get_epmc_annotations_to_file(pmcids)

100%|█████████████████████████████████████████| 300/300 [03:20<00:00,  1.49it/s]


In [4]:
annotations_df = pd.read_csv('../data/annotations_from_api.csv', sep='\t', names=['pmcid', 'exact', 'token', 'ner'])
annotations_df

,pmcid,exact,token,ner
0,PMC4792959,"Plant Biology, 260 Panama Street, Stanford, C",Panama,Organisms
1,PMC4792959,"In plants, such barriers can either act before",plants,Organisms
2,PMC4792959,her act before (pre-pollination barriers) or a...,pollination,Gene Ontology
3,PMC4792959,barriers) or after pollination (post-pollinat...,pollination,Gene Ontology
4,PMC4792959,or after pollination (post-pollination barrie...,pollination,Gene Ontology
...,...,...,...,...
88185,PMC3458065,f erythrocytes from HIV-positive individual,HIV,Organisms
88186,PMC3458065,lts suggest that in HIV-positive individual,HIV,Organisms
88187,PMC3458065,ing high amounts of HIV by the presence of,HIV,Organisms
88188,PMC3458065,-gp160/120 on their membranes and this may produc,membranes,Gene Ontology


In [8]:
annotations_df['ner'].unique()

array(['Organisms', 'Gene Ontology', 'Diseases', 'Gene_Proteins',
       'Experimental Methods', 'Chemicals', 'Accession Numbers',
       'Resources'], dtype=object)

In [9]:
annotations_df_other = annotations_df[annotations_df['ner'].isin(['Gene Ontology', 'Experimental Methods', 'Accession Numbers', 'Resources'])]
annotations_df_other

,pmcid,exact,token,ner
2,PMC4792959,her act before (pre-pollination barriers) or a...,pollination,Gene Ontology
3,PMC4792959,barriers) or after pollination (post-pollinat...,pollination,Gene Ontology
4,PMC4792959,or after pollination (post-pollination barrie...,pollination,Gene Ontology
5,PMC4792959,Pre-pollination barriers can be spatial or tem...,pollination,Gene Ontology
7,PMC4792959,"ecies, whereas post-pollination barriers come ...",pollination,Gene Ontology
...,...,...,...,...
88126,PMC3458065,ated for 7 to 10 days and assayed for syncytiu...,syncytium formation,Gene Ontology
88127,PMC3458065,is able to inhibit syncytium formation (loss ...,syncytium formation,Gene Ontology
88134,PMC3458065,end-point dilution assay in MT-2 cells cultu,assay,Experimental Methods
88156,PMC3458065,e IgG anti-HIV binding to erythrocyte membrane.,membrane,Gene Ontology


In [17]:
# # Get unique PMCIDs
# unique_pmcids = annotations_df_other['pmcid'].unique()url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
# unique_pmcids

# import requests
# from bs4 import BeautifulSoup
# import re

# def get_full_text_xml(pmcid):
#     url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
#     response = requests.get(url)
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'xml')
#         p_tags = soup.find_all('p')
#         p_texts = [tag.get_text() for tag in p_tags]
#         return p_texts
#     else:
#         return None

In [12]:
unique_pmcids = annotations_df_other['pmcid'].unique()

In [10]:
def get_full_text_xml(pmcid):
    path = f"../data/300_articles_source_files/{pmcid}.xml"
    try:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        soup = BeautifulSoup(content, 'lxml-xml')  # Use lxml-xml parser
        plain_tags = soup.find_all('plain')
        plain_texts = [tag.get_text() for tag in plain_tags]
        return plain_texts
    except FileNotFoundError:
        print(f"File {pmcid}.xml not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [17]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

def check_previous_word_pos(sentence, target_word):
    # Process the sentence to get tokenized words and their POS tags
    doc = nlp(sentence)
    
    # Find the target word and get the POS of the previous word
    for i, token in enumerate(doc):
        if token.text == target_word and i > 0:
            previous_word = doc[i - 1]
            pos_of_previous = previous_word.pos_
            
            # Check if the POS is one of the ones we want to avoid
            if pos_of_previous in ['DET', 'ADV', 'VERB', 'ADP']:
                return False  # POS to be avoided
            else:
                return True  # Acceptable POS

    return False  # Return False if the target word is not found or is the first word


In [37]:
def find_sentence_with_substring(string_list, substring):
    for text in string_list:
        sentences = re.split(r'(?<=[.!?])\s+', text)
        for sentence in sentences:
            if substring in sentence:
                return sentence
    return None

def check_previous_word_pos(sentence, prev_word):
    # Process the sentence to get tokenized words and their POS tags
    doc = nlp(sentence)
    
    for token in doc:
        if token.text == prev_word:
            if token.i > 0:
                pos_of_previous = doc[token.i - 1].pos_
                return pos_of_previous not in ['DET', 'ADV', 'VERB', 'ADP', 'NUM', 'CONJ', 'SCONJ']

    return False

def adjust_token_for_context(sentence, token, ner_):
    check_list = ['the', 'an', 'a', 'that', 'after', 'and', 'total', 'using', 'by','as', 'for', 'in', 'with', 
                  'underwent', 'of', 'with', 'but', 'or', 'before', 'at', 
                  'activate', 'research', 'favors', 'prevent', 'to', 'drive', 'on']
    specific_phrases = ['viral capture assay', 'flow cytometry assay', 'luminescent cell viability assay']
    
    
    words = sentence.split()
    punctuation_marks = [',', '.', '?', '!']
    extended_tokens = set()
    
   
    for phrase in specific_phrases:
        # Find all case-insensitive occurrences of the phrase
        found_phrases = re.findall(phrase, sentence, re.IGNORECASE)
        # Add each found phrase to the extended tokens
        for found in found_phrases:
            extended_tokens.add(found)
            
    # Remove parentheses from the token
    token_cleaned = token.replace('(', '').replace(')', '')

    # Adjust token for larger term if token is a substring and does not contain specific punctuation
    if ner_ in ['Gene Ontology', 'Experimental Methods']:
        for word in words:
            cleaned_word = word.replace('(', '').replace(')', '')
            if token_cleaned in cleaned_word and cleaned_word != token_cleaned and not any(mark in cleaned_word for mark in punctuation_marks):
                extended_tokens.add(cleaned_word)

    # Check for words immediately before the token and add those combinations
    for i, word in enumerate(words):
        cleaned_word = word.replace('(', '').replace(')', '')
        if cleaned_word == token_cleaned and i > 0:
            prev_word = words[i - 1].replace('(', '').replace(')', '')
            if token_cleaned.lower() in ['assay', 'assays', 'insertion', 'insertions', 'elisa']:
                if prev_word.lower() not in check_list and (prev_word.isupper() or not prev_word.islower()):
                    extended_tokens.add(prev_word + ' ' + token_cleaned)
            elif prev_word.lower() not in check_list and check_previous_word_pos(sentence, prev_word):
                extended_tokens.add(prev_word + ' ' + token_cleaned)

    # Always add the token itself, after removing parentheses
    extended_tokens.add(token_cleaned)

    return list(extended_tokens)




def process_pmcid(df, pmcid, p_texts):
    sentences_data = {}
    for _, row in df[df['pmcid'] == pmcid].iterrows():
        sentence = find_sentence_with_substring(p_texts, row['exact'])
        if sentence:
            if sentence not in sentences_data:
                sentences_data[sentence] = set()

            updated_tokens = adjust_token_for_context(sentence, row['token'], row['ner'])
            for updated_token in updated_tokens:
                sentences_data[sentence].add((updated_token, row['ner']))

    return [[pmcid, sentence, list(ner_tags)] for sentence, ner_tags in sentences_data.items()]

final_data = []

for pmcid in tqdm(unique_pmcids):
    p_texts = get_full_text_xml(pmcid)
    if p_texts:
        processed_data = process_pmcid(annotations_df_other, pmcid, p_texts)
        final_data.extend(processed_data)

# Convert to DataFrame
final_df = pd.DataFrame(final_data, columns=['pmcid', 'sentence', 'ner'])

# Save as TSV
final_df.to_csv('../data/xxx7.tsv', sep='\t', index=False)

100%|█████████████████████████████████████████| 297/297 [01:15<00:00,  3.93it/s]


In [42]:
import ast

def calculate_spans(sentence, entities):
    spans = []
    for entity, etype in entities:
        start = sentence.find(entity)
        if start != -1:
            end = start + len(entity)
            spans.append([start, end, entity, etype])  # List instead of tuple
    return spans

def resolve_overlaps(spans):
    spans.sort(key=lambda x: (x[1] - x[0]), reverse=True)  # Sort by span length
    resolved = []
    for span in spans:
        overlap = False
        for r in resolved:
            if not (span[1] <= r[0] or span[0] >= r[1]):  # Check for overlap
                overlap = True
                break
        if not overlap:
            resolved.append(span)
    return resolved



In [44]:
df = pd.read_csv('../data/xxx7.tsv', sep = '\t')
df['new_ner'] = df.apply(lambda row: resolve_overlaps(calculate_spans(row['sentence'], ast.literal_eval(row['ner']))), axis=1)
df.drop('ner', axis=1, inplace=True)
df.rename(columns={'new_ner': 'ner'}, inplace=True)
df.to_csv('../data/xxx8.tsv', sep = '\t', index=False)

In [45]:
# Rename entity names in file2.csv as per the provided mapping
entity_rename_map = {
    'Gene Ontology': 'GO',
    'Experimental Methods': 'EM', 
    'Accession Numbers': 'AN',
    'Resources': 'RS'
}

file2_df = pd.read_csv('../data/xxx8.tsv', sep = '\t')
# Applying the renaming
file2_df['ner'] = file2_df['ner'].apply(lambda x: eval(x))
for row in file2_df['ner']:
    for entity in row:
        entity[3] = entity_rename_map.get(entity[3], entity[3])

# Checking the changes
file2_df.head()


,pmcid,sentence,ner
0,PMC4792959,"In plants, such barriers can either act before...","[[96, 112, post-pollination, GO], [48, 63, pre..."
1,PMC4792959,Pre-pollination barriers can be spatial or tem...,"[[145, 161, post-pollination, GO], [0, 15, Pre..."
2,PMC4792959,The latter usually act at the genomic level (f...,"[[212, 230, cell communication, GO]]"
3,PMC4792959,Whereas barriers that prevent fertilization (b...,"[[22, 43, prevent fertilization, GO], [50, 65,..."
4,PMC4792959,"In plants, successful fertilization starts wit...","[[11, 35, successful fertilization, GO]]"


In [53]:
def parse_ner_string(ner_string):
    """
    Parse the NER string into a list of entities. If the NER string is 'None', return an empty list.
    """
    if ner_string == "['None']":
        return []
    return eval(ner_string)

def resolve_overlapping_entities(entities):
    """
    Resolve overlapping entities based on the provided priority order.
    """
    priority_order = ['GP', 'DS', 'OG', 'CD', 'AN', 'RS', 'EM', 'GO']
    sorted_entities = sorted(entities, key=lambda x: (x[0], priority_order.index(x[3])))
    resolved_entities = []

    for entity in sorted_entities:
        if not resolved_entities or entity[0] > resolved_entities[-1][1]:
            resolved_entities.append(entity)

    return resolved_entities



In [54]:
import glob

def process_and_enrich_csv(file_path, file2_df):
    # Read the CSV file
    file1_df = pd.read_csv(file_path, names=['sentence', 'ner'])
    # Parse NER column
    file1_df['ner'] = file1_df['ner'].apply(parse_ner_string)

    # Combine entities from both files and resolve overlaps
    enriched_entities = []
    for index, row in file1_df.iterrows():
        sentence_entities = row['ner']
        # Find matching sentences in file2 and combine entities
        matches = file2_df[file2_df['sentence'] == row['sentence']]
        for _, match in matches.iterrows():
            sentence_entities.extend(match['ner'])
        # Resolve overlapping entities
        resolved_entities = resolve_overlapping_entities(sentence_entities)
        enriched_entities.append(resolved_entities)

    # Create a new DataFrame for the enriched data
    enriched_df = file1_df.copy()
    enriched_df['ner'] = enriched_entities
    return enriched_df

# Path to the folder containing CSV files
folder_path = '../data/CD_GP_DS_OG_test/*.csv'

# Process each CSV file in the folder
for file_path in glob.glob(folder_path):
    enriched_df = process_and_enrich_csv(file_path, file2_df)
    enriched_file_path = file_path.replace('clean_', 'enriched_')
    enriched_df.to_csv(enriched_file_path, index=False)

print("Processing complete.")


Processing complete.


In [49]:
file1_df = pd.read_csv('../data/CD_GP_DS_OG_test/clean_CD_GP_DS_OG_train.csv', names=['sentence', 'ner'])
# Parse NER column in file1
file1_df['ner'] = file1_df['ner'].apply(parse_ner_string)

file1_df.head()

,sentence,ner
0,GR3027 antagonizes GABAA receptor-potentiating...,"[[135, 149, hyperammonemia, DS], [117, 121, ra..."
1,M.,[]
2,Johansson and A.,[]
3,Agusti contributed equally to this work.,[]
4,Hepatic encephalopathy (HE) is one of the prim...,"[[67, 82, liver cirrhosis, DS], [0, 22, Hepati..."


,sentence,ner
0,GR3027 antagonizes GABAA receptor-potentiating...,"[[0, 6, GR3027, CD], [19, 33, GABAA receptor, ..."
1,M.,[]
2,Johansson and A.,[]
3,Agusti contributed equally to this work.,[]
4,Hepatic encephalopathy (HE) is one of the prim...,"[[0, 22, Hepatic encephalopathy, DS], [24, 26,..."
